In [1]:
import numpy
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import pickle
import logging
from tensorflow.python.client import device_lib
import sys
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

/home/ubuntu/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
with open('TedAllTalksString.pickle.dat', 'rb') as pickle_file:
    raw_text = pickle.load(pickle_file)
raw_text = raw_text.lower()
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print( "Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model

Total Characters:  9152671
Total Vocab:  75
Total Patterns:  9152571


In [ ]:
device_lib.list_local_devices()


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 884801637461990218, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 357892096
 locality {
   bus_id: 1
 }
 incarnation: 6620346960521662940
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"]

In [ ]:
args={'num_gpus':8}
gpu_list = ["gpu(%d)" % i for i in range(args['num_gpus'])]

    
model = Sequential()
model.add(LSTM(400, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(200))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

filename='Tester/weights-improvement-02-1.6788.hdf5'
model.load_weights(filename)

model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
# backend_agnostic_compile(model, 'categorical_crossentropy',optimizer='adam', metrics=['accuracy'], args=args)
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=50, batch_size=128, callbacks=callbacks_list)

Epoch 1/50
7608192/9152571 [=======================>......] - ETA: 1:03:35 - loss: 1.5953

In [ ]:
def backend_agnostic_compile(model, loss, optimizer, metrics, args):
    if keras.backend._backend == 'mxnet':
        gpu_list = ["gpu(%d)" % i for i in range(args['num_gpus'])]
        model.compile(loss=loss,
          optimizer=optimizer,
          metrics=metrics, 
          context = gpu_list)
    else:
        if args['num_gpus'] > 1:
            print("Warning: num_gpus > 1 but not using MxNet backend")
        model.compile(loss=loss,
          optimizer=optimizer,
          metrics=metrics)

In [25]:
import mxnet

In [ ]:
keras.backend._backend